## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import pandas as pd
import numpy as np


TensorFlow 2.x selected.


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [2]:
from google.colab import drive
from pandas import read_csv

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = read_csv('/content/drive/My Drive/prices.csv')

### Check all columns in the dataset

In [5]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data=data.drop(['date','symbol'],axis=1)

In [7]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [8]:
data_df = data.head(1000)
data_df.shape

(1000, 5)

In [9]:
data_df['volume'] = data_df['volume']/1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
data_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X = data_df.iloc[:,0:4]
Y = data_df.iloc[:,4]
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

In [12]:
train_x.dtypes

open     float64
close    float64
low      float64
high     float64
dtype: object

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
features = tf.math.l2_normalize(train_x)

In [15]:
train_y.shape

(700,)

In [16]:
features.shape

TensorShape([700, 4])

In [0]:
train_y=train_y.reshape(-1,1)

In [18]:
train_y.shape

(700, 1)

In [19]:
train_x.shape

(700, 4)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(features, W, b):
    y_pred = tf.add(tf.matmul(features, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg


4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [24]:
for i in range(100):
    i=i+5    
    W, b = train(features, train_y, W, b)
    print('Current Training Loss on iteration', i, loss(train_y, prediction(features, W, b)))

('Current Training Loss on iteration', 5, <tf.Tensor: id=127, shape=(), dtype=float32, numpy=249.30289>)
('Current Training Loss on iteration', 6, <tf.Tensor: id=133, shape=(), dtype=float32, numpy=248.08824>)
('Current Training Loss on iteration', 7, <tf.Tensor: id=139, shape=(), dtype=float32, numpy=246.92168>)
('Current Training Loss on iteration', 8, <tf.Tensor: id=145, shape=(), dtype=float32, numpy=245.8014>)
('Current Training Loss on iteration', 9, <tf.Tensor: id=151, shape=(), dtype=float32, numpy=244.72556>)
('Current Training Loss on iteration', 10, <tf.Tensor: id=157, shape=(), dtype=float32, numpy=243.69232>)
('Current Training Loss on iteration', 11, <tf.Tensor: id=163, shape=(), dtype=float32, numpy=242.69995>)
('Current Training Loss on iteration', 12, <tf.Tensor: id=169, shape=(), dtype=float32, numpy=241.74701>)
('Current Training Loss on iteration', 13, <tf.Tensor: id=175, shape=(), dtype=float32, numpy=240.83179>)
('Current Training Loss on iteration', 14, <tf.Tenso

### Get the shapes and values of W and b

In [25]:
W.shape

TensorShape([4, 1])

In [26]:
b.shape

TensorShape([1])

### Model Prediction on 1st Examples in Test Dataset

In [27]:
pred = prediction(test_x,W,b)
pred

<tf.Tensor: id=731, shape=(300, 1), dtype=float32, numpy=
array([[18.377804 ],
       [ 8.660276 ],
       [10.569506 ],
       [ 7.101538 ],
       [12.465557 ],
       [11.339802 ],
       [13.652492 ],
       [11.934232 ],
       [ 6.307893 ],
       [ 7.5755186],
       [19.601715 ],
       [ 9.496658 ],
       [ 7.7977667],
       [ 7.659108 ],
       [ 8.627894 ],
       [ 7.1240325],
       [ 8.431783 ],
       [ 8.470331 ],
       [ 6.3010955],
       [19.12101  ],
       [18.933086 ],
       [18.619598 ],
       [19.247036 ],
       [ 7.101456 ],
       [20.18671  ],
       [10.149591 ],
       [ 8.075562 ],
       [20.566021 ],
       [19.92317  ],
       [ 7.6014137],
       [ 8.606036 ],
       [19.7034   ],
       [19.897865 ],
       [11.692717 ],
       [11.07741  ],
       [19.621323 ],
       [18.32882  ],
       [12.808182 ],
       [12.884718 ],
       [ 7.069028 ],
       [ 8.479572 ],
       [ 6.321325 ],
       [11.93749  ],
       [11.228573 ],
       [ 7.3516912

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_df = read_csv('/content/drive/My Drive/11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [29]:
iris_df.head()


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [30]:
iris_df = pd.get_dummies(iris_df, 'Species')
iris_df.head()


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [31]:
new_iris_df=iris_df.drop('Id',axis=1)
new_iris_df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [0]:

X_1 = new_iris_df.iloc[:,0:4]
Y_1 = new_iris_df.iloc[:,4:7]

### Splitting the data into feature set and target set

In [0]:
from tensorflow.keras.datasets import boston_housing

#(trainX2, trainY2), (testX2, testY2) = boston_housing.load_data(test_split=0.3)
trainX2, testX2, trainY2, testY2 = train_test_split(X_1, Y_1, test_size=0.2)

In [34]:
print(trainX2.shape)
print(trainY2.shape)
print(testX2.shape)
print(testY2.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

# Normalize the data
model.add(tf.keras.layers.BatchNormalization())

# Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [37]:
model.fit(trainX2, trainY2, validation_data=(testX2, testY2), epochs=50,
          batch_size = trainX2.shape[0])

Train on 120 samples, validate on 30 samples
Epoch 1/50
120/120 [==============================] - 1s 5ms/sample - loss: 2.2155 - accuracy: 0.0167 - val_loss: 4.4596 - val_accuracy: 0.2333
Epoch 2/50
120/120 [==============================] - 0s 49us/sample - loss: 2.1173 - accuracy: 0.0167 - val_loss: 4.1390 - val_accuracy: 0.2333
Epoch 3/50
120/120 [==============================] - 0s 39us/sample - loss: 2.0262 - accuracy: 0.0250 - val_loss: 3.8467 - val_accuracy: 0.2333
Epoch 4/50
120/120 [==============================] - 0s 34us/sample - loss: 1.9416 - accuracy: 0.0250 - val_loss: 3.5805 - val_accuracy: 0.2333
Epoch 5/50
120/120 [==============================] - 0s 33us/sample - loss: 1.8628 - accuracy: 0.0250 - val_loss: 3.3382 - val_accuracy: 0.2333
Epoch 6/50
120/120 [==============================] - 0s 35us/sample - loss: 1.7896 - accuracy: 0.0250 - val_loss: 3.1178 - val_accuracy: 0.2333
Epoch 7/50
120/120 [==============================] - 0s 36us/sample - loss: 1.7213 - 

### Model Prediction

In [43]:
aa = model.predict(testX2)
aa

array([[0.08026927, 0.573506  , 0.3462247 ],
       [0.12205211, 0.47481844, 0.4031295 ],
       [0.19587961, 0.48404488, 0.32007557],
       [0.20489587, 0.5112398 , 0.28386432],
       [0.69535154, 0.2049045 , 0.09974398],
       [0.22145183, 0.46512842, 0.31341973],
       [0.24866907, 0.46201935, 0.28931156],
       [0.1337578 , 0.5310629 , 0.3351793 ],
       [0.81132686, 0.13626564, 0.05240743],
       [0.72600096, 0.19032991, 0.08366913],
       [0.11050531, 0.523598  , 0.36589667],
       [0.17800175, 0.5022513 , 0.31974688],
       [0.75652117, 0.17500529, 0.06847355],
       [0.74142927, 0.17805268, 0.08051808],
       [0.08240402, 0.50722975, 0.41036624],
       [0.18975905, 0.46013737, 0.35010356],
       [0.23047386, 0.46232235, 0.30720383],
       [0.7053332 , 0.19822031, 0.09644652],
       [0.17536595, 0.5325347 , 0.2920994 ],
       [0.08946394, 0.44177037, 0.4687657 ],
       [0.09418073, 0.4460621 , 0.45975718],
       [0.18932818, 0.4994024 , 0.31126934],
       [0.

### Save the Model

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 4)                 16        
_________________________________________________________________
batch_normalization_1 (Batch (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 3)                 15        
Total params: 47
Trainable params: 31
Non-trainable params: 16
_________________________________________________________________


In [48]:
model.predict(testX2)

array([[0.08026927, 0.573506  , 0.3462247 ],
       [0.12205211, 0.47481844, 0.4031295 ],
       [0.19587961, 0.48404488, 0.32007557],
       [0.20489587, 0.5112398 , 0.28386432],
       [0.69535154, 0.2049045 , 0.09974398],
       [0.22145183, 0.46512842, 0.31341973],
       [0.24866907, 0.46201935, 0.28931156],
       [0.1337578 , 0.5310629 , 0.3351793 ],
       [0.81132686, 0.13626564, 0.05240743],
       [0.72600096, 0.19032991, 0.08366913],
       [0.11050531, 0.523598  , 0.36589667],
       [0.17800175, 0.5022513 , 0.31974688],
       [0.75652117, 0.17500529, 0.06847355],
       [0.74142927, 0.17805268, 0.08051808],
       [0.08240402, 0.50722975, 0.41036624],
       [0.18975905, 0.46013737, 0.35010356],
       [0.23047386, 0.46232235, 0.30720383],
       [0.7053332 , 0.19822031, 0.09644652],
       [0.17536595, 0.5325347 , 0.2920994 ],
       [0.08946394, 0.44177037, 0.4687657 ],
       [0.09418073, 0.4460621 , 0.45975718],
       [0.18932818, 0.4994024 , 0.31126934],
       [0.

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [50]:
model.save('R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: R6_Internal_Lab_UpdatedTF2_Prices_Iris.ipynb/assets
